Importing Libraries

In [ ]:
import nltk, re, string
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pickle

Pre-Processing of the tweets (Data)

In [ ]:
def process_tweet(tweet):
  stemmer = nltk.PorterStemmer()
  stopwords_english = stopwords.words("english")
  tweet = re.sub(r'\$\w', '', tweet)
  tweet = re.sub(r'^RT[\s]+', '', tweet)
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
  tweet = re.sub(r'#', '', tweet)
  tokenizer = nltk.TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
  tweet_tokens = tokenizer.tokenize(tweet)

  tweet_clean=[]
  for word in tweet_tokens:
    if(word not in stopwords_english and
       word not in string.punctuation):
      stem_word = stemmer.stem(word)
      tweet_clean.append(stem_word)

  return tweet_clean

In the preprocessing following things happen:
1. re function removes all the unnecessary number, links, special char etc. from the tweets
2. tokenizer funct takes string as an input and return the list of words.
3. a loop is used for further cleaning by using the stemmer function which return the base words of the string eg.(eating --> eat)

In [ ]:
def build_freqs(tweets, ys):
  #convert np array ito lists since zip needs an iterable
  #the sqeeze function is necessary or the list ends up with one element
  #Also note that this is just a NOP is ys is already a list
  yslist = np.squeeze(ys).tolist()

  #start with an empty dictionary and populate it by looping over all tweets
  # and over all processed words in each tweets.

  freqs={}
  for y, tweet in zip(yslist, tweets):
    for word in process_tweet(tweet):
      pair = (word, y)
      if pair in freqs:
        freqs[pair] +=1
      else:
        freqs[pair] =1
  return freqs



the build_freq() function contains a empty dictionary which will contain all the words with there class lable "positive" or "negetive" along with the no. of times it occured. 

In [ ]:
nltk.download('twitter_samples')
nltk.download('stopwords')
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]
train_x = train_pos + train_neg
test_x = test_pos + test_neg
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)
freqs = build_freqs(train_x, train_y)

print('type(freqs) =' + str(type(freqs)))
print('len(freqs) =' + str(len(freqs.keys())))


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


type(freqs) =<class 'dict'>
len(freqs) =11339


In this cell the data set are being prepared for the training and testing purpose 
1. all_positive_tweets and all_negative_tweets variable store the respective tweets.
2. test and train positive and negative data are divided 
3. x variable of training data if both postive and negative is initialized 
4. y-target variable is initialized by combining positive and negative labels
5. freqs dictionary is been created using build_freqs function.
6. type and length is printed 


# **Model building** - Logistic Regression Model

In [ ]:
# sigmoid function
def sigmoid(z):
  zz= np.negative(z)
  h = 1 / (1 + np.exp(zz))
  return h

In [ ]:
#cost function and Gradient

def gradientDescent(x, y, theta, alpha, num_iters):
  '''
  Input:
        x: matrix of the features which is (m+1)
        y: corresponding labels of the input matrix x, dimension (m,1)
        theta: weight vector of dimension (n+1, 1):
        alpha: learning rate
        num_iters: number of interations you want to train your model for
  Output:
        J: final cost
        theta: your final weight vector
  '''
  #get 'm', the number of rows in matrix x
  m= x.shape[0]
  for i in range(0, num_iters):
    z= np.dot(x,theta)
    h = sigmoid(z)
    #calculate cost func
    cost = -1/m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(), np.log(1-h)))
    #update the weight theta
    theta = theta - (alpha/m) * np.dot(x.transpose(), (h-y))
  cost=float(cost)
  return cost, theta

Feature Extraction:

In [ ]:
def extract_features(tweet, freqs):
  '''
  Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple(word, label)
  Output:
        x: a feature vector of dimension (1,3)
  '''
  word_l = process_tweet(tweet)
  x=np.zeros((1,3))
  #bias term is set to 1
  x[0,0] = 1
  for word in word_l:
    #increment the word count for the positive label 1
    x[0,1] += freqs.get((word, 1.0), 0)
    #increment the word count for the negative label 0
    x[0,2] += freqs.get((word, 0.0), 0)

  assert (x.shape ==(1,3))
  return x


Testing training data:

In [ ]:
tmp1 = extract_features(train_x[22], freqs)
print(tmp1)

[[1.000e+00 3.006e+03 1.240e+02]]


Testing training data explaination:
-> Usually we get a dataset with lot of features/columns, here we just have text data.
-> those 3 numbers are the feature set that we have built using build_freqs() and extract_feature() function.
-> bulid_freq() builts a dictionary having words as keys and number of times they have occured in corpus values.
-> Extract feature takes in sum of these values for positive and negetive words, i.e. tmp1[1] and tmp[2]

How this feature will be used to predict the losgistic Regression:
-> First a hypothesis is buiold in which for our case will be h(x) = b1 + b2*x1 + b3*x2
-> here b1=1, b2 and b3 are determined by cost and gradient descent function, x1 and x2 are the positive and negetive words feature set.

In [ ]:
#Training the model

#collect the features "x" and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
  X[i, :]= extract_features(train_x[i], freqs)

#training labels corresponding to X
Y = train_y

#Apply gradient descent 
#these values are predefined
J, theta = gradientDescent(X, Y, np.zeros((3,1)), 1e-9, 1500)



In [ ]:
def predict_tweet(tweet, freqs, theta):
  """
  Input:
        tweet: a string 
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights 
  Output:
        y_pred: the probablity of a tweet being positive or negetive 
  """
  # extract the feature of the tweet and store it into x
  x= extract_features(tweet, freqs)
  y_pred= sigmoid(np.dot(x, theta))

  return y_pred
  

In [ ]:
def test_logistic_regression(test_x, test_y, freqs, theta):
  """
  Input:
        test_x: a list of tweets
        test_y: (n,1) vector with the coresponding label for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3,1)
  Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
  """
  # list for storing predictions
  y_hat=[]

  for tweet in test_x:
    #get the label prediction for the tweet
    y_pred = predict_tweet(tweet, freqs, theta)
    if y_pred > 0.5:
      y_hat.append(1)
    else:
      y_hat.append(0)
  accuracy = (y_hat == np.squeeze(test_y)).sum() / len(test_x)

  return accuracy


In [ ]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy: .4f}")

Logistic regression model's accuracy =  0.9950


In [ ]:
#prediction with own tweet
def pre(sentence):
  yhat= predict_tweet(sentence, freqs, theta)
  if yhat> 0.5:
    return "Positive statement"
  elif yhat == 0:
    return "Neutral statement"
  else:
    return "Negative statement"



In [ ]:
my_tweet = 'Riya is a very bad Girl'
res = pre(my_tweet)
print(res)

Negative statement
